In [ ]:
import pandas as pd
import numpy as np


In [ ]:
# # Original import:

# # mbti dataset
# mbti = pd.read_csv("/content/myers-briggs-data.csv")
# mbtis = pd.read_csv("/content/myer-briggs-data.cvs.csv")

# # ONET datasets
# onet_salaries = pd.read_csv("/content/onet_data_with_salaries.csv")
# onet_humn = pd.read_csv("/content/onet_data_with_human_characteristics.csv")



# MBTI dataset (CSV)
mbti = pd.read_csv("/content/drive/MyDrive/Dataset/myer-briggs-data.csv")
#mbtis = pd.read_csv("/content/myer-briggs-data.cvs.csv")

# ONET datasets (Excel files, so use read_excel)
onet_salaries = pd.read_excel("/content/drive/MyDrive/Dataset/onet_data_with_salaries.xlsx")
onet_humn = pd.read_excel("/content/drive/MyDrive/Dataset/onet_data_with_human_characteristics.xlsx")

#Added:
onet_complete = pd.read_excel("/content/drive/MyDrive/Dataset/complete_onet_data_with_human_characteristics.xlsx")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Dataset/myer-briggs-data.csv'

In [ ]:
###TASK 1: Dataset Inspection###
#inspecting the datasets
def inspect_dataset(df,name="Dataset"):
  print(f"/n====={name} INSPECTION=====")

#looking through shape and info about the datasets
  print(f"Shape: {df.shape}")
  print(f"Columns: {df.columns}")
  print("Info:")
  print(df.info())

  print("\nHead:")
  print(df.head())
#missing values
  print("\nMissing Values per Column:")
  print(df.isnull().sum())

#summary statistics
  print("\nSummary Statistics:(numeric columns):")
  print(df.describe())

  #for checking outliers
  print("\nOutlier Check (based on IQR, numeric columns):")
  numeric_cols = df.select_dtypes(include=[np.number]).columns
  for col in numeric_cols:
      q1 = df[col].quantile(0.25)
      q3 = df[col].quantile(0.75)
      iqr = q3 - q1

      outliers = df[(df[col] < (q1 - 1.5 * iqr)) | (df[col] > (q3 + 1.5 * iqr))]
      print(f"{col}: {len(outliers)} potential outliers")

  print("="*50)

# Inspecting all the datasets
inspect_dataset(mbti, "MBTI Dataset")
inspect_dataset(mbtis, "MBTI (duplicate?) Dataset")
inspect_dataset(onet_salaries, "O*NET Salaries Dataset")
inspect_dataset(onet_humn, "O*NET Human Characteristics Dataset")








###TASK 2: Standardization Diagnostics:###
# Step 1: Align Column Names
def standardize_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )
    return df

onet_salaries = standardize_columns(onet_salaries)
onet_humn = standardize_columns(onet_humn)
onet_complete = standardize_columns(onet_complete)

# Reorder columns to match complete dataset (base schema)
onet_salaries = onet_salaries.reindex(columns=onet_complete.columns.intersection(onet_salaries.columns))
onet_humn = onet_humn.reindex(columns=onet_complete.columns)

# Step 2: Identify Redundancy
def compare_columns(base, other, name):
    base_cols = set(base.columns)
    other_cols = set(other.columns)

    missing = base_cols - other_cols
    extra = other_cols - base_cols

    print(f"\n Schema check for {name}:")
    if missing:
        print(f"  - Missing columns: {missing}")
    else:
        print("  - No missing columns compared to base.")
    if extra:
        print(f"  - Extra columns: {extra}")
    else:
        print("  - No extra columns compared to base.")

compare_columns(onet_complete, onet_salaries, "O*NET Salaries")
compare_columns(onet_complete, onet_humn, "O*NET Human Characteristics")


print(" The other ONET datasets are redundant once alignment is done.")

# Step 3: Drop Duplicates
# Keep only the complete dataset as the single source of truth
onet_standardized = onet_complete.drop_duplicates()

# Save Cleaned Dataset
output_path = "/content/drive/MyDrive/Dataset/onet_standardized.csv"
onet_standardized.to_csv(output_path, index=False)

print("\n Cleaning pipeline complete.")
print(" Using `complete_onet_data_with_human_characteristics` as the single source of truth.")
print(f" Final shape: {onet_standardized.shape} (rows, columns)")
print(f" Final dataset saved at: {output_path}")

# Preview first rows
print("\n Preview of standardized ONET dataset:")
display(onet_standardized.head())




NameError: name 'mbti' is not defined

In [20]:
### TASK 3: Clean data, remove stopwords and tokenize text ###
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mbti = pd.read_csv("/content/drive/MyDrive/myer-briggs-data.csv")

import re
import string
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(ENGLISH_STOP_WORDS)

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()  # instead of word_tokenize
    tokens = [w for w in tokens if w not in stop_words]
    return " ".join(tokens)

mbti['cleaned_text'] = mbti['posts'].apply(clean_text)

# TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2), min_df=5)
X = vectorizer.fit_transform(mbti['cleaned_text'])
y = mbti['type']

mbti

# Split the 'posts' column by '|'
#mbti_expanded = mbti.assign(posts=mbti['posts'].str.split('|')).explode('posts')

# Clean the text again
#mbti_expanded['cleaned_text'] = mbti_expanded['posts'].apply(clean_text)

#mbti_expanded.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,type,posts,cleaned_text
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,intj moments sportscenter plays prankswhat lif...
1,ENTP,'I'm finding the lack of me in these posts ver...,im finding lack posts alarmingsex boring posit...
2,INTP,'Good one _____ https://www.youtube.com/wat...,good course say know thats blessing cursedoes ...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",dear intp enjoyed conversation day esoteric ga...
4,ENTJ,'You're fired.|||That's another silly misconce...,youre firedthats silly misconception approachi...
...,...,...,...
8670,ISFP,'https://www.youtube.com/watch?v=t8edHB_h908||...,just think cats fi doms reason websites haven ...
8671,ENFP,'So...if this thread already exists someplace ...,soif thread exists someplace does heck delete ...
8672,INTP,'So many questions when i do these things. I ...,questions things purple pill pick winning lott...
8673,INFP,'I am very conflicted right now when it comes ...,conflicted right comes wanting children honest...


In [ ]:
###TASK 4: Categorical Encoding###
